In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>", fig.align = "center"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

# Additional dependencies for this notebook
install.packages("reticulate")

packageVersion("Voyager")

# Introduction

In this introductory vignette for the [`SpatialFeatureExperiment`](https://bioconductor.org/packages/devel/bioc/html/SpatialFeatureExperiment.html) data representation and [`Voyager`](https://bioconductor.org/packages/devel/bioc/html/Voyager.html) analysis package, we demonstrate a basic exploratory data analysis (EDA) of spatial transcriptomics data. Basic knowledge of R and [`SingleCellExperiment`](https://bioconductor.org/packages/release/bioc/html/SingleCellExperiment.html) is assumed.

This vignette showcases the packages with a Visium spatial gene expression system dataset. The technology was chosen due to its popularity, and therefore the availability of numerous publicly available datasets for analysis [@Moses2022-xz].  

![Bar chart showing the number of institutions using each spatial transcriptomics data collection method. Only methods used by at least 3 different institutions are shown. The bars are colored by category of the methods. 10X Visium which is based on sequence barcoding is used by over 100 institutions. Following Visium are GeoMX DSP and GeoMX WTA which are "regions of interest" (ROI) methods, along with 2016 ST and some other ROI selection methods.](https://pachterlab.github.io/LP_2021/04-current_files/figure-html/n-insts-1.png)
While Voyager was developed with the goal of facilitating the use of geospatial methods in spatial genomics, this introductory vignette is restricted to non-spatial scRNA-seq EDA with the Visium dataset. For a vignette illustrating univariate spatial analysis with the same dataset, see the more advanced [exploratory spatial data analyis vignette](https://pachterlab.github.io/voyager/articles/vig2_visium.html) with the same dataset.

Here we load the packages used in this vignette.

In [ ]:
library(Voyager)
library(SpatialFeatureExperiment)
library(SingleCellExperiment)
library(SpatialExperiment)
library(scater)
library(scran)
library(patchwork)
library(bluster)
library(SFEData)
library(BiocParallel)
library(stringr)
library(ggplot2)
library(sparseMatrixStats)
library(dplyr)
library(reticulate)
library(concordexR)
library(BiocNeighbors)
theme_set(theme_bw(10))

In [ ]:
# Specify Python version to use gget
PY_PATH <- Sys.which("python")
use_python(PY_PATH)
py_config()

In [ ]:
system("pip3 install gget")

In [ ]:
gget <- import("gget")

# Mouse skeletal muscle dataset
The dataset used in this vignette is from the paper [Large-scale integration of single-cell transcriptomic data captures transitional progenitor states in mouse skeletal muscle regeneration](https://doi.org/10.1038/s42003-021-02810-x) [@McKellar2021-ek]. Notexin was injected into the tibialis anterior muscle of mice to induce injury, and the healing muscle was collected 2, 5, and 7 days post injury for Visium analysis. The dataset in this vignette is from the timepoint at day 2. The vignette starts with a `SpatialFeatureExperiment` (SFE) object.

The gene count matrix was directly downloaded [from GEO](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM4904759). All 4992 spots, whether in tissue or not, are included. The tissue boundary was found by thresholding the H&E image in OpenCV, and small polygons were removed as they are likely to be debris. Spot polygons were constructed with the spot centroid coordinates and diameter in the Space Ranger output. The `in_tissue` column in `colData` indicates which spot polygons intersect the tissue polygons, and is based on `st_intersects()`. 

Tissue boundary, nuclei, myofiber, and Visium spot polygons are stored as `sf` data frames in the SFE object. The Visium spot polygons are called "spotPoly" in this SFE object. The `SpatialFeatureExperiment` package has a few convenience wrappers to get and set common types of geometries, including `spotPoly()` for Visium (or other technologies when relevant) spot polygons, `cellSeg()` for cell segmentation, `nucSeg()` for nuclei segmentation, and `centroids()` for cell centroids. Behind the scene are specially named `sf` data frames. See [the vignette of `SpatialFeatureExperiment`](https://bioconductor.org/packages/devel/bioc/vignettes/SpatialFeatureExperiment/inst/doc/SFE.html) for more details on the structure of the SFE object. 

The SFE object of this dataset is provided in the `SFEData` package; we begin by downloading the data and loading it into R.

In [ ]:
(sfe <- McKellarMuscleData("full"))

The authors provided the full resolution hematoxylin and eosin (H&E) image on GEO, which we downsized to facilitate its display:
![A cross section of mouse muscle is slightly off center to the lower left. In the middle of the tissue is the notexin injury site with leukocyte infiltration and fewer myofibers. The rest of the tissue section is tightly packed with myofibers.](https://raw.githubusercontent.com/pachterlab/voyager/documentation/vignettes/tissue_lowres_5a.jpeg)

The image can be added to the SFE object and plotted behind the geometries, and needs to be flipped to align to the spots because the origin is at the top left for the image but bottom left for geometries.

In [ ]:
sfe <- addImg(sfe, file = "tissue_lowres_5a.jpeg", sample_id = "Vis5A", 
              image_id = "lowres", 
              scale_fct = 1024/22208)
sfe <- mirrorImg(sfe, sample_id = "Vis5A", image_id = "lowres")

# Quality control
## Spots
We begin quality control (QC) by plotting various metrics both as violin plots and in space. The QC metrics are pre-computed and stored in `colData` (for spots) and `rowData` of the SFE object.

In [ ]:
names(colData(sfe))

Below we plot the total unique molecular identifier (UMI) counts per spot. The commented out line of code shows how to compute the total UMI counts. The `maxcell` argument is the maximum number of pixels to plot for the image; the image is downsampled if it has more pixels than `maxcells`. This can speed up plotting when plotting the same image in multiple facets.

In [ ]:
# colData(sfe)$nCounts <- colSums(counts(sfe))
violin <- plotColData(sfe, "nCounts", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top")
spatial <- plotSpatialFeature(sfe, "nCounts", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary", 
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) +
    theme_void()
violin + spatial

Some spots in the injury site with leukocyte infiltration have high total counts. Spatial autocorrelation of the total counts is apparent, which will be discussed in a later section of this vignette.

Next we find number of genes detected per spot. The commented out line of code shows how to find the number of genes detected.

In [ ]:
# colData(sfe)$nGenes <- colSums(counts(sfe) > 0)
violin <- plotColData(sfe, "nGenes", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top")
spatial <- plotSpatialFeature(sfe, "nGenes", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) +
    theme_void()
violin + spatial

As commonly done for scRNA-seq data, here we plot nCounts vs. nGenes

In [ ]:
plotColData(sfe, x = "nCounts", y = "nGenes", colour_by = "in_tissue")

This plot has two branches for the spots in tissue, which turn out to be related to myofiber size. See [the exploratory spatial data analysis (ESDA) Visium vignette](https://pachterlab.github.io/voyager/articles/vig2_visium.html).

As is commonly done for scRNA-seq data, we plot the proportion of mitochondrially encoded counts. The commented out code shows how to find this proportion:

In [ ]:
# mito_ind <- str_detect(rowData(sfe)$symbol, "^Mt-")
# colData(sfe)$prop_mito <- colSums(counts(sfe)[mito_ind,]) / colData(sfe)$nCounts
violin <- plotColData(sfe, "prop_mito", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top")
spatial <- plotSpatialFeature(sfe, "prop_mito", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) +
    theme_void()
violin + spatial

As expected, spots outside tissue have a higher proportion of mitochondrial counts, because when the tissue is lysed, mitochondrial transcripts are are less likely to degrade than cytosolic transcripts as they are protected by a double membrane. However, spots on myofibers also have a high proportion of mitochondrial counts, because of the function of myofibers. The injury site with leukocyte infiltration has a lower proportion of mitochondrial counts.

To see the relationship between the proportion of mitochondrial counts and total UMI counts,  we plot them against each other as is commonly done in scRNA-seq analysis to identify low quality cells, i.e. cells with few UMI counts and a high proportion of mitochondrial counts.

In [ ]:
plotColData(sfe, x = "nCounts", y = "prop_mito", colour_by = "in_tissue")

There are two clusters for the spots in tissue, which also turn out to be related to myofiber size. See the [ESDA Visium vignette](https://pachterlab.github.io/voyager/articles/vig2_visium.html). 

So far we haven't seen spots that are obvious outliers in these QC metrics.

The following analyses only use spots in tissue, which are selected as follows:

In [ ]:
sfe_tissue <- sfe[, colData(sfe)$in_tissue]
sfe_tissue <- sfe_tissue[rowSums(counts(sfe_tissue)) > 0,]

## Genes
As in scRNA-seq, gene expression variance in Visium measurements is overdispersed compared to variance of counts that are Poisson distributed.

To understand the mean-variance relationship, we compute the mean, variance, and coefficient of variance (CV2) for each gene among spots in tissue:

In [ ]:
rowData(sfe_tissue)$means <- rowMeans(counts(sfe_tissue))
rowData(sfe_tissue)$vars <- rowVars(counts(sfe_tissue))
# Coefficient of variance
rowData(sfe_tissue)$cv2 <- rowData(sfe_tissue)$vars/rowData(sfe_tissue)$means^2

To avoid overplotting and better show point density on the plot, we use a 2D histogram. The color of each bin indicates the number of points in that bin.

In [ ]:
plotRowDataBin2D(sfe, "means", "vars", bins = 50) +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal()

The red line, $y = x$ is what is expected for Poisson distributed data, but we find that the variance is higher for more highly expressed genes than expected from Poisson distributed counts. The coefficient of variation shows the same.

In [ ]:
plotRowDataBin2D(sfe, "means", "cv2", bins = 50) +
    geom_abline(slope = -1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal()

# Normalize data

We demonstrate the use of `scater` for normalization below, although we note that it is not necessarily the best approach to normalizing spatial transcriptomics data. The problem of when and how to normalize spatial transcriptomics data is non-trivial because, as the `nCounts` plot in space shows above, spatial autocorrelation is evident. Furthemrore, in Visium, reverse transcription occurs in situ on the spots, but PCR amplification occurs after the cDNA is dissociated from the spots. Artifacts may be subsequently introduced from the amplification step, and these would not be associated with spatial origin. Spatial artifacts may arise from the diffusion of transcripts and tissue permeablization. However, given how the total counts seem to correspond to histological regions, the total counts may have a biological component and hence should not be treated as a technical artifact to be normalized away as in scRNA-seq data normalization methods. In other words, the issue of normalization for spatial transcriptomics data, and Visium in particular, is complex and is currently unsolved.

There is no one way to normalize non-spatial scRNA-seq data. The commented out code implements the `scran` method [@Lun2016-yq]. To simplify the matter, we only perform `logNormCounts()` in this introductory vignette. 

Note that `scater`'s `logNormCounts()` is quite different from that in Seurat. Let $N$ denote the total UMI count in one Visium spot, $\bar N$ the average total UMI count in all spots in this dataset, and $x$ denote the UMI count of one gene in the Visium spot of interest. Seurat performs log normalization as $\mathrm{log}\left( \frac{x}{N/10000} + 1 \right)$, where the natural log is used. In contrast, with default parameters, `scater` uses $\mathrm{log_2}\left( \frac{x}{N/\bar N} + 1 \right)$. The pseudocount (default to 1), library size factors (default to $N/\bar N$), and transform (default to log2) can be changed. Log 2 is used because differences in values can be interpreted as log fold change.

In [ ]:
# clusters <- quickCluster(sfe_tissue)
# sfe_tissue <- computeSumFactors(sfe_tissue, clusters=clusters)
# sfe_tissue <- sfe_tissue[, sizeFactors(sfe_tissue) > 0]
sfe_tissue <- logNormCounts(sfe_tissue)

Next, we identify highly variable genes (HVGs), which will be used for principal component analysis (PCA) dimensionality reduction. Again, there are different ways to identify HVGs, and `scater` does so differently from Seurat. In both frameworks, the log normalized data is used by default. In summary, in Seurat, with default parameters, a Loess curve is fitted to the log transformed data (the log normalized data is log transformed for fitting purposes), and the fitted values are exponentiated as the expected variance for each gene. Then the expected variance and the mean are used to standardize the log normalized gene expression; the standardized values are used to calculate a standardized variance for each gene. The top HVGs are the genes with the largest standardized variance.

In `scater`, with default parameters, a parametric non-linear curve of variance vs. mean for each gene of the log normalized data. Then the log ratio of the actual variance to the fitted variance from the curve calculated, and a Loess curve is fitted to this log ratio vs. mean for each gene. The "technical" component of the variance is the fitted values from the Loess curve. The "biological" component is the difference between the actual variance and the Loess fitted variance. The top HVGs are genes with the largest biological component. See the documentation of `modelGeneVar()`, `fitTrendVar()`, and `getTopHVGs()` for more details.

These differences can lead to different downstream results. While we don't comment on which way is better in this vignette, it's important to be aware of such differences.

In [ ]:
dec <- modelGeneVar(sfe_tissue)
hvgs <- getTopHVGs(dec, n = 2000)

# Dimension reduction and clustering

In [ ]:
sfe_tissue <- runPCA(sfe_tissue, ncomponents = 30, subset_row = hvgs,
                     scale = TRUE) # scale as in Seurat

In [ ]:
ElbowPlot(sfe_tissue, ndims = 30)

In [ ]:
plotDimLoadings(sfe_tissue, dims = 1:4, swap_rownames = "symbol")

Do the clustering to show on the dimension reduction plots

In [ ]:
colData(sfe_tissue)$cluster <- clusterRows(reducedDim(sfe_tissue, "PCA")[,1:3],
                                           BLUSPARAM = SNNGraphParam(
                                               cluster.fun = "leiden",
                                               cluster.args = list(
                                                   resolution_parameter = 0.5,
                                                   objective_function = "modularity")))

In [ ]:
plotPCA(sfe_tissue, ncomponents = 3, colour_by = "cluster")

The principal components (PCs) can be plotted in space. Due to spatial autocorrelation of many genes and spatial regions of different histological characters, even though spatial information is not used in the PCA procedure, the PCs may show spatial structure. 

In [ ]:
spatialReducedDim(sfe_tissue, "PCA", ncomponents = 4, 
                  colGeometryName = "spotPoly", divergent = TRUE, 
                  diverge_center = 0, image = "lowres", maxcell = 5e4)

PC1, which explains far more variance than PC2, separates the injury site leukocytes and myofibers close to the site from the Visium myofibers. PC2 highlights the center of the injury site and some myofibers near the edge. PC3 highlights the muscle tendon junctions. PC4 does not seem to be informative; it might have picked up an outlier.

It is also possible to run UMAP following the PCA, as is done for scRNA-seq. We do not recommend producing a UMAP since the procedure distorts distances, and does not respect either local or global structure in the data [@Chari2021-hb]. However, for completeness, we show how to compute a UMAP below:

In [ ]:
set.seed(29)
sfe_tissue <- runUMAP(sfe_tissue, dimred = "PCA", n_dimred = 3)

In [ ]:
plotUMAP(sfe_tissue, colour_by = "cluster")

UMAP is often used to visualize clusters. An alternative to UMAP is [`concordex`](https://bioconductor.org/packages/devel/bioc/html/concordexR.html), which quantitatively shows the proportion of neighbors on the k nearest neighbor graph with the same cluster label. To be consistent with the default in igraph Leiden clustering, we use k = 10.

In [ ]:
g <- findKNN(reducedDim(sfe_tissue, "PCA")[,1:3], k = 10)
res <- calculateConcordex(g$index, labels = sfe_tissue$cluster, k = 10, 
                          return.map = TRUE)

The cluster labels are permuted to estimate a null distribution, and the observed values can be compared to the simulated values:

In [ ]:
plotConcordexSim(res)

The observed value is much higher than in the simulated values, indicating good clustering. The single number is the average of all clusters. Values for different clusters can be plotted in a heatmap:

In [ ]:
heatConcordex(res, angle_col = 0, cluster_rows = FALSE, cluster_cols = FALSE)

The diagonal represents the proportion of neighbors of cells from each cluster that are from the same cluster. That the off diagonal entries are very low indicate good clustering.

More interesting in spatial transcriptomics, is to locate the clusters in space, and this can be done as follows:

In [ ]:
plotSpatialFeature(sfe_tissue, "cluster", colGeometryName = "spotPoly",
                   image = "lowres")

While spatial information is not explicitly used in clustering, due to spatial autocorrelation of gene expression and the histological regions, some of these clusters are spatially contiguous. There are many methods to find spatially informed clusters, such as [`BayesSpace`](https://bioconductor.org/packages/release/bioc/html/BayesSpace.html) [@Zhao2021-vb], which is on Bioconductor.

Remark on spatial regions: In geographical space, there is usually no one single way to define spatial regions. For example, influenced by both sociology and geology, LA county can be partitioned into regions such as Eastside, Westside, South Central, San Fernado Valley, San Gabriel Valley, Pomona Valley, Gateway Cities, South Bay, and etc., each containing multiple smaller cities or parts of LA City, each of which can be further divided into many neighborhoods, such as Koreatown, Highland Park, Lincoln Heights, and etc. Definitions of some of these regions are subject to dispute. Meanwhile, LA county can also be partitioned into watersheds of the LA River, San Gabriel River, Ballona Creek, and etc., as well as different rock formations. Which kind of spatial region at which resolution is relevant depends on the question being asked. There are also gray areas in spatial regions. For example, the Whittier Narrows dam intercepts both the San Gabriel River and Rio Hondo (a large tributary of the LA River), so whether the dam area belongs to the watershed of San Gabriel River or LA River is unclear. 

Similarly, in spatial transcriptomics, while methods identifying spatial regions currently generally only aim to give one result, multiple results at different resolutions depending on the question asked may be relevant. Furthermore, methods for spatial region demarcation to be used for spatial -omics would ideally provide uncertainty assessments for assignment of cells or Visium spots. An existing geospatial method that accounts for such uncertainty is [`geocmeans`](https://cran.r-project.org/web/packages/geocmeans/) [@Zhao2013-tw], which is on CRAN.

In both the geographical and histological space, there conflicting views on spatial variation. On the one hand, methods that identify spatially variable genes such as SpatialDE often assume that gene expression vary smoothly and continuously in space. On the other hand, methods identifying spatial regions attempt to identify discrete regions. The continuous variation in features might be why definitions of geographical neighborhoods are often subject to dispute. Some existing methods attempt to harmonize the two views. For example, the spatially variable gene method [`belayer`](https://github.com/raphael-group/belayer) [@Ma2022-dc] takes discrete tissue layers into account.

# Non-spatial differential expression
Cluster marker genes can be found using differential analysis methods as is commonly done for scRNA-seq. Below is an example with the Wilcoxon rank sum test:

In [ ]:
markers <- findMarkers(sfe_tissue, groups = colData(sfe_tissue)$cluster,
                       test.type = "wilcox", pval.type = "all", direction = "up")

The result is sorted by p-values:

In [ ]:
markers[[1]]

We can use the [gget enrichr](https://pachterlab.github.io/gget/enrichr.html) module from the [gget](https://pachterlab.github.io/gget/) package to perform a gene enrichment analysis. You can choose from >200 enrichment databases which are listed on the [Enrichr website](https://maayanlab.cloud/Enrichr/#libraries). Here, we are analyzing the top 20 genes for cluster 1 using the default ontology database [GO_Biological_Process_2021](http://geneontology.org/):

In [ ]:
enrichr_genes <- rownames(markers[[1]])[1:20]
gget_e <- gget$enrichr(enrichr_genes, ensembl=TRUE, database = "ontology")

# Plot results of gene enrichment analysis
# Count number of overlapping genes
gget_e$overlapping_genes_count <- lapply(gget_e$overlapping_genes, length) |> as.numeric()
# Only keep the top 10 results
gget_e <- gget_e[1:10,]
gget_e |>
    ggplot() +
    geom_bar(aes(
        x = -log10(adj_p_val),
        y = reorder(path_name, -adj_p_val)
    ),
    stat = "identity",
  	fill = "lightgrey",
  	width = 0.5,
    color = "black") +
    geom_text(
        aes(
            y = path_name,
            x = (-log10(adj_p_val)),
            label = overlapping_genes_count
        ),
        nudge_x = 0.25,
        show.legend = NA,
        color = "red"
    ) +
  	geom_text(
        aes(
            y = Inf,
            x = Inf,
      			hjust = 1,
      			vjust = 1,
            label = "# of overlapping genes"
        ),
        show.legend = NA,
        size=4,
        color = "red"
    ) +
    geom_vline(linetype = "dashed", linewidth = 0.5, xintercept = -log10(0.05)) +
    ylab("Pathway name") +
    xlab("-log10(adjusted P value)")

Significant markers for each cluster can be obtained as follows:

In [ ]:
genes_use <- vapply(markers, function(x) rownames(x)[1], FUN.VALUE = character(1))
plotExpression(sfe_tissue, rowData(sfe_tissue)[genes_use, "symbol"], x = "cluster",
               colour_by = "cluster", swap_rownames = "symbol")

We'll use the module [gget info](https://pachterlab.github.io/gget/info.html) to get additional information on these genes, such as their descriptions, synonyms, transcripts and more from a collection of reference databases including [Ensembl](https://ensembl.org/), [UniProt](https://www.uniprot.org/), and [NCBI](https://www.ncbi.nlm.nih.gov/). Here, we are showing their gene descriptions from [NCBI](https://www.ncbi.nlm.nih.gov/):

In [ ]:
gget_info <- gget$info(genes_use)

rownames(gget_info) <- gget_info$primary_gene_name
select(gget_info, ncbi_description)

These genes are interesting to view in spatial context:

In [ ]:
plotSpatialFeature(sfe_tissue, genes_use, colGeometryName = "spotPoly", ncol = 3,
                   image = "lowres", maxcell = 5e4, swap_rownames = "symbol")

# Moran's I
Tobler's first law of geography states that

> Everything is related to everything else. But near things are more related than distant things.

This observation motivates the examination of spatial autocorrelation. Positive spatial autocorrelation is evident when nearby things tend to be similar, such as that weather in Pasadena and downtown Los Angeles (as opposed to the weather in Pasadena and San Francisco). Negative spatial autocorrelation is evident when nearby things tend to be more dissimilar, like squares on a chessboard. Spatial autocorrelation can arise from an intrinsic process such as diffusion or communication by physical contact, or result from a covariate that has such an intrinsic process, or in areal data, when the areal units of observation are smaller than the scale of the spatial process.

The most commonly used measure of spatial autocorrelation is Moran's I, defined as

$$
I = \frac{n}{\sum_{i=1}^n \sum_{j=1}^n w_{ij}} \frac{\sum_{i=1}^n \sum_{j=1}^n w_{ij} (x_i - \bar{x})(x_j - \bar{x})}{\sum_{i=1}^n (x_i - \bar{x})^2},
$$

where $n$ is the number of spots or locations, $i$ and $j$ are different locations, or spots in the Visium context, $x$ is a variable with values at each location, and $w_{ij}$ is a spatial weight, which can be inversely proportional to distance between spots or an indicator of whether two spots are neighbors, subject to various definitions of neighborhood and whether to normalize the number of neighbors. The [`spdep`](https://r-spatial.github.io/spdep/index.html) package uses the neighborhood.

Moran's I is similar to the Pearson correlation between the value at each location and the average value at its neighbors (but not identical, see [@Lee2001-tm]). Just like Pearson correlation, Moran's I is generally bound between -1 and 1, where positive value indicates positive spatial autocorrelation and negative value indicates negative spatial autocorrelation. 
Spatial dependence analysis in `spdep` requires a spatial neighborhood graph. The graph for adjacent Visium spot can be found with

In [ ]:
colGraph(sfe_tissue, "visium") <- findVisiumGraph(sfe_tissue)

We mentioned that spatial autocorrelation is apparent in total UMI counts. Here's what Moran's I shows:

In [ ]:
calculateMoransI(t(colData(sfe_tissue)[,c("nCounts", "nGenes")]), 
                 listw = colGraph(sfe_tissue, "visium"))

K means kurtosis. The positive values of Moran's I indicate positive spatial autocorrelation.

## Spatially variable genes
A spatially variable gene is a gene whose expression depends on spatial locations, rather than being spatially random, like salt grains spread on a soup. Spatially variable genes can be identified by spatial autocorrelation signatures, and sometimes Moran's I is used to compare and assess spatially variable genes identified with different methods. Below `BPPARAM` is used to paralelize the computation of Moran's I for 2000 highly variable genes, and 2 cores are used with the SNOW backend.

In [ ]:
sfe_tissue <- runMoransI(sfe_tissue, features = hvgs, colGraphName = "visium",
                         BPPARAM = SnowParam(2))

The results are stored in `rowData`

In [ ]:
rowData(sfe_tissue)

The `NA`'s are for genes that are not highly variable and Moran's I was not computed for those genes. We rank the genes by Moran's I and plot them in space as follows:

In [ ]:
df <- rowData(sfe_tissue)[hvgs,]
ord <- order(df$moran_Vis5A, decreasing = TRUE)
df[ord, c("symbol", "moran_Vis5A")]

We see that some genes that have strong positive spatial autocorrelation, but don't observe strong negative spatial autocorrelation.

Let's get some additional information on the genes with the strongest positive spatial autocorrelation in space using [gget info](https://pachterlab.github.io/gget/info.html) as above:

In [ ]:
gget_info2 <- gget$info(rownames(df)[1:6])

rownames(gget_info2) <- gget_info2$primary_gene_name
select(gget_info2, ncbi_description)

Let's plot these genes:

In [ ]:
plotSpatialFeature(sfe_tissue, rownames(df)[1:6], colGeometryName = "spotPoly",
                   image = "lowres", maxcell = 5e4, swap_rownames = "symbol")

These genes do indeed look spatially variable. However, such spatial variability can simply be due to the histological regions in space, or in other words, spatial distribution of different cell types. There are many methods to identify spatially variable genes, often involving Gaussian process modeling, which are far more complex than Moran's I, such as [`SpatialDE`](https://bioconductor.org/packages/release/bioc/html/spatialDE.html) [@Svensson2018-sx]. However, such methods usually don't account for the histological regions, except for `C-SIDE` [@Cable2022-ma], which identifies spatially variable genes within cell types. This leads to the question of what is really meant by "cell type". It remains to see how spatial methods made specifically for identifying spatially variable genes compare with methods that don't explicitly use spatial information but simply perform differential analysis between cell types which often are in spatially defined histological regions. 

Another consideration in using Moran's I is the extent to which the strength of spatial autocorrelation varies in space. What if a gene exhibits strong spatial autocorrelation in one region, but not in another? Should the different histological regions be analyzed separately in some cases? 

There are ways to see whether Moran's I is statistically significant, and many other methods to explore spatial autocorrelation. These are discussed in the more advanced [ESDA Visium vignette](https://pachterlab.github.io/voyager/articles/vig2_visium.html). 

# Session Info

In [ ]:
sessionInfo()

# References